In [1]:
import glob
import os
import pandas as pd
from flask import Flask, request, render_template, session, send_from_directory, redirect, url_for, send_file, flash
from werkzeug.utils import secure_filename
import re
import os
from tqdm import tqdm
import pandas as pd
import openpyxl
import numpy as np
import nltk
from nltk.corpus import stopwords
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
def rm(path):
    os.remove(path)


def background_remove(path: object) -> object:
    task = Process(target=rm(path))
    task.start()


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS


def clean(i):
    i=re.sub(r'\w*\d\w*',' ',i)
    i=re.sub('\d+',' ',i)
    i=re.sub(r'[\.\-_\)@*#&;$\(:+/%,^=~`"]',' ',i)
    i=re.sub(r'[^A-Za-z]',' ',i)
    i=re.sub('\s+',' ',i)
    i=re.sub('\t',' ',i)
    i=re.sub('\n',' ',i)
#     i=re.sub('[A-Z]{5}[0-9]{17}',' ',i)
    return i

def replace_fun(x):
    x = re.sub("[^\d\.]","",str(x))
    x = re.sub('\s+','',x)
    x=re.sub('\t',' ',x)
    return x

UPLOAD_FOLDER = "C:\\Users\\SAL005\\Desktop\\FUND_FLOW_HOSTED\\uploads"
ALLOWED_EXTENSIONS = 'xlsx'
DOWNLOAD_FOLDER = "C:\\Users\\SAL005\\Desktop\\FUND_FLOW_HOSTED\\data_folder"

app = Flask(__name__, template_folder='C:\\Users\\SAL005\\Desktop\\FUND_FLOW_HOSTED\\templates')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['DOWNLOAD_FOLDER'] = DOWNLOAD_FOLDER




@app.route("/")
def home():
    return render_template('index_2.html')

@app.route('/upload', methods=['GET','POST'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            map_name = str(filename).split('.')[0]
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            df = pd.read_excel(os.path.join(UPLOAD_FOLDER, filename))
            df['Description']= df['Description'].astype(str)
            df['Description']=df['Description'].map(lambda x: x.upper())
            df['Description_clean']= df['Description'].copy()
            df['Description_clean']= df['Description_clean'].apply(clean)
            df['Description_clean']=df['Description_clean'].str.replace('PVT','PRIVATE')
            df['Description_clean']=df['Description_clean'].str.replace('LTD','LIMITED')
            df['Transaction Mode'] = ''
            org_names_final1=[]
            for idx,x in tqdm(enumerate(df['Description_clean'])):
                if (' RTGS ' not in x and ' NEFT ' not in x and 'CASH ' not in x and 'FEE ' in x or 'FEE' in x or 'CHGS ' in x or 'CHG ' in x or ' CHARGE ' in x or 'CHRG ' in x or 'CHRGS ' in x or ' CHRGS' in x or 'CHARGES ' in x or ' CHARGES' in x or 'CGS ' in x or 'CHGS ' in x or 'GST ' in x or 'FEE ' in x or 'AMC ' in x or 'MAINTENANCE ' in x or 'MIN BAL ' in x or 'CHARG ' in x or 'SURCHARGE' in x or 'LIEN' in x or ' CHARGE ' in x or 'CASH HANDLING' in x) == True:
                    df['Transaction Mode'].iloc[idx] = 'CHARGES'
                elif ('CASH ' in x or 'BY CASH' in x and 'CHGS ' not in x and 'CHG ' not in x or 'PAID TO SELF' in x and ' CHARGE ' not in x and 'CHRG ' not in x and 'CGS ' not in x and 'CHGS ' not in x and 'TDS ' not in x and 'GST ' not in x and 'CDM' in x) == True:
                    df['Transaction Mode'].iloc[idx] = 'CASH'
                elif (' RTGS ' not in x and ' NEFT ' not in x and 'CASH ' not in x and 'FEE ' in x or 'FEE' in x or 'CHGS ' in x or 'CHG ' in x or ' CHGARGE ' in x or 'CHRG ' in x or 'CHRGS ' in x or ' CHRGS' in x or 'CHARGES ' in x or ' CHARGES' in x or 'CGS ' in x or 'CHGS ' in x or 'GST ' in x or 'FEE ' in x or 'AMC ' in x or 'MAINTENANCE ' in x or 'MIN BAL ' in x or 'CHARG ' in x or 'SURCHARGE' in x or 'LIEN' in x or ' CHARGE ' in x or 'CASH HANDLING' in x) == True:
                    df['Transaction Mode'].iloc[idx] = 'CHARGES'
                elif ('RTGS ' in x or 'RTGSO ' in x or 'RTGSR ' in x or ' RTGS ' in x or 'RTGS' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='RTGS'
                elif ('NEFT ' in x or ' NEFT ' in x or 'NEFT' in x or 'BULK NEFT' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='NEFT'
                elif ('INTT' in x or 'INT ' in x or 'SBINT' in x or 'INT PD' in x or 'INT COLL' in x or 'INT TO' in x or 'INT FOR' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='INTEREST'
                elif ('UPI ' in x or 'UPIAB ' in x or ' UPIN' in x or 'UPIAR ' in x or ' UPI' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='UPI'
                elif ('AUTO SWEEP' in x or 'AUTO SWAP' in x or 'AUTOSWAP' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='AUTO SWEEP'
                elif ('IMPS' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='IMPS'
                elif ('RETURN' in x or 'RETD' in x or 'REJECT' in x or 'REVERSE' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='RETURN/REVERSE/REJECT'
                elif ('TRFR ' in x or 'BY TR ' in x or 'EPAY' in x or 'VPS' in x or 'FT CR ' in x or 'FT DR ' in x or 'TO TR ' in x or 'TR ' in x or 'TRF' in x or 'TFR' in x or 'TRANSFER' in x or 'TRANSFER' in x or 'BILINFT' in x or 'INFT' in x or 'INBIFT' in x or 'INF' in x or 'MOB' in x or 'INB ' in x or 'TO ' in x or ' TO ' in x or 'BIL' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='TRANSFER'
                elif ('BY CLEARING' in x or ' CLY ' in x or 'BY CH'in x or 'BY CHQ' in x or 'OW CHQ' in x or 'INWCLG' in x or 'CHQ' in x or 'CH' in x or 'CREDITED' in x or 'MICR' in x or 'INWD' in x or 'CLG' in x or 'CHQS' in x or 'CHW' in x or 'INST' in x or 'CHEQUE' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='CHEQUE'
                elif ('DD' in x or 'DEMAND DRAFT' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='DEMAND DRAFT'
                elif ('MSWIPE' in x or ' POS ' in x or 'POS ' in x or 'MID ' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='POS'
                elif ('ACH ' in x or 'ECH ' in x or 'NACH ' in x or 'CMS ' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='ACH/NACH/CMS'
                elif ('TAX' in x or 'TDS' in x or 'CBDT' in x or 'CGST' in x or 'UGST' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='TAX'
                elif ('FIXED DEPOSIT' in x or 'FD' in x or 'TD ' in x or 'TFD ' in x or 'FDR ' in x or 'FDRS' in x or 'PROCEEDS' in x or 'CBDT' in x and df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='FIXED DEPOSIT'
                elif (df['Transaction Mode'].iloc[idx]=='') == True:
                    df['Transaction Mode'].iloc[idx]='OTHERS'
            
            stops=['STATE BAN','NEW CA ATM','EPC RETURN PAY','NETBANK','ATM CORPORATION','CCTC','NETBANK MUM','SWIFT CAR','STATE BANK OF','TPARTY','HDFC BANK','ORIENTAL BANK','BANK LT','DD ISSUE','PAID','DD NUM','CANCELLATION','CANCLN','CLEARING','CHQ NO','ASPER LIST',' REJ ','OW CHQ','STATE ','CREDITED ENTRY RVRSD',' IW CHQ','IW ','CREDITED','DAY  LAT','DAY LATENCY','LATENCY',' OUTWARD ','OUTWARD','CANCELLATION OF ','ICICIBAN','DENA','NRTGS','TO ',' DIF','BDBL','BDBLR','FEES',' DIFFER','STOPPED BY DRAWER','STOPPED BY D',' ARRANGEMENT',' DIFF','HAND ','RECEIPT','OW REJ',' RATN ','CTS','REFER TO DRAWER','INVESTMENT AND RETURNS','ERR ENTRY','INSTRUMENT','OUTDATED','REVERSED CH NO','NAME','MICR','INW',' WBO ','CANCL','CANCELLED','OW WBO','OWN','DD CC','IDFBH','TRF','ICICH','DD CANCELLED','DD ',' ICI ','AXS','KOT ','MC ','AXSK','COMMISSION','DD NUM PAID','DD CANCELLATION','DD CANCL','IRM','IOI PAYMENT CHARGES','REGISTRATION CHARGES','MMT','ARRANGEME','BY TH','DIF ','EXCEEDS','REFER TO DRAWER/N','DIFFERS','ALTERATION REQUIRED','DRAWERS SIGNATURE TO OPER','PAYMENT STOPPED B','ISSUED','RETURN','ALTERATION REQUIRED','DRAWEE BANK P','OPERATE AC NOT','NOT DRAWN ON US','DRAWN AMOUNT IN','NAME DIFFERS ON ADVICE','DRAWEE BANK','DRAWERS SIGNATURE D','ARRANGEMENT','DIFFERS','PAYMENT STOPPED BY THE','DRAWERS SIGNATURE DIFF','PAYMENT STOPPED BY TH','INSTRUMENT UNDATED W','INF ','SDMC','FROM PH ST','FROM PH','FROM ','OKI ','ORBC','SYNB','BY TRASNFER','INSTA KIT NO','REJECT','FUNDS INSUFFICIENT','DRAWEE','OTHER REASONS','PLEAS','ON ADVICE','CHQ RETURN','MICR CTS','DRAWERS SIGNATURE','DIF','MAHB','ICICIQ','ICICIH','DR ',' ICICI ','ICIC','SBILT','HDFC B','HDFC','BANKING',' BAN ','  NA ','OW RTN','ICICIBAN','MIN BAL SERV CHGS','ACH TP ACH HOME','ACH TP ACH HOME CA','STATE BANK OF','SAK ','CHQ DEPOSIT','CLG CHQ','CORR OFF SYSTEM CASH WITHDRAWA','ECS ','UPI ','CHQ DISHONOUR','POSPUR ','BLPGCM DT','FRM BANK OF MAHARASHTRA IMPS XXXXXXX','IMPS XXXXXXX','FRM BANK OF MAHARASHTRA','ACH ','IMPS ','SLABWISE NMMB','IOC REF NO','RTGS ','UTIBR ','BRN UTIBH','HDFC BANK','YES BANK LIMITED','PUNJAB SIND BANK','IDBI BANK LIMITED','ATM RD','AC ARN','DEBIT','FRM BANK','IOBAR','HDF','ATM CARD','CR ','INET ','PSIB ','YBL','B B D DF E B F A','UNION BANK OF INDIA','RTGSO ','TRF ','UBINR','IPAY ','INST ','AXISF ','INB','SBIN','CBINR','CA OTHERS','KKBKH','NEFT ','AXMB','BARBV','BARBQ','PUNBH','CBINH','BARBS','PUNBH','SDC','P M ','P A ','PAYTM PAY UPI','STATE BAN PAYMENT','ICICI BAN UPI',' FEDERAL B UPI','CANARA BA UPI','PAYTM PAY NA',' PUNJAB NA UPI','MAHARASHT UPI','STATE BAN UPI',' INDIAN BA UPI','AXIS BANK UPI','DBS BANK PAYMENT',' IDFC FIRS UPI','KOTAK MAH UPI','YESBANK Y UPI','UNION BAN UPI',' BANK OF I UPI',' DBS BANK UPI',' CITIBANK UPI',' KARNATAKA UPI','CENTRAL B UPI','INDIA POS UPI','FEDERAL B NA',' INDIAN OV UPI',' STATE BAN NA',' UNION BAN PAYMENT',' PAYTMPA X ',' YES BANK UPI',' ICICI BAN OK UPI','AH AIRTEL PA UPI',' BANK OF B BIN','PUNJAB NA NA',' BANK OF B NA','AXIS BANK PAYMENT',' EQUITAS S PAYMENT','IDBI BANK PAYMENT',' PAYTM PAY PAYMENT','ICICI BAN USDT',' INDUSIND PAYMENT',' INDIAN OV NOTE LI','KOTAK MAH PAYMENT','BANK OF B PAYMENT',' UPI','ICICI BAN INR TO','PAYMENT','IDFC FIRS PAYMENT','BANK OF B UPI','MAHARASHT PAYMENT','CANARA BA PAYMENT','ICICI BAN PAYMENT','PUNJAB NA PAYMENT','ICICI BAN PAY TO','KOTAK MAH NA','KARNATAKA PAYMENT','KARNATA X ','KARNATAK IMPSP A','KARNATAK TRANSFER','FEDERAL B SENT VI','IDFC FIRS BAG','KOTAK MAH ','ICICI BAN PAY','YES BANK PAYMENT',' IDBI BANK UPI',' BANK OF B PAYMENT','UNION BAN SEND PA',' IDFC FIRS PAYMENT','BANK OF B UPI',' BANK OF M UPI','BANK OF B MITTAL','AXIS BANK NA','AIRTEL PA',' AXIS BANK TEA',' TRANSFE ',' KOTAKMA X ','MAHARASHT PAYMENT','BANK OF I PAYMENT','FEDERAL B PAYMENT','KOTAK MAH COFEE','BANK OF B',' IDFC FIRS ','YES BANK ','PUNJAB NA ','FINO PAYM',' ICICI BAN ',' CANARA BA ','FEDERAL B ',' IDBI BANK','KARNATAKA','MAHARASHT',' UCO BANK ','IDFC FIRS ','ICICIBA X ',' YES BANK ','INDUSIND','PAYT ','MBSENTTOB ','PAY ','OKAXIS',' ICI','ICICI U','OKICICI','AXISBANK','OKAXIS','ICICI PAY','OKHD','OKSBI US','CUSTOM ','TRANSFER TO ','PAYTM','UBIN','AXL','KOTAK','OKCBANK','KBL','MY KOTAK KKBK ','KKBK ','MY  KKBK ','IFSC','NPCI','IBL','SBI','MY AXIS UTIB','IDFB','PUNB','TRFR FROM','NFS CASH WDL','MABCHGS','HSBC','INT PD TO','BARBW','STAX','POS ','SMS ','OTHPG','INDB','BY CLG','CLG','BRN  PAID TO','POS ','H D F C BANK L','PUNBH', 'SBIN', 'CITIN', 'KKBH', 'BARBS', 'AXMB', 'SBIN', 'MR ', 'MRS ', 'INF ', 'UTR NO', 'IBKL', 'HSBC', 'HSBCN', 'ICICI', 'KVBL', 'ICIC', 'YESB', 'REJ FC REJECT', 'NEFT DR', 'NEFT CR', 'UTIB', 'NEF', 'FUNDTRANSFERTO', 'TRANSFER FROM', 'IB FUND TRANSFER', 'TPTTRANSFER', 'TRANSFER TO', 'TR ', 'FR ', 'TR FROM', 'FT DR', 'DT CR', 'BY PNB CH', 'VCH NO', 'BANK', 'OTH', 'OUT ', 'OUTWRD', 'BOB', 'BT TRF', 'BY ', 'TR ', 'CHEQUES', 'AXIS BANK', 'BY CH NO', 'SBI', 'PNB', 'SBH', 'UCO', 'UNI ', 'UPC', 'UTI', 'INST', 'OW CLG', 'ZONE', 'NMICR ', 'CTO', 'LATENCY', 'CTO', 'LOCAL', 'BRN OW RTN  REJECT EXCEEDS ARRANGEME', 'BRN OW RTN  REJECT FUNDS INSUFFICIENT', 'RTN', 'REJECT', 'H D F C', 'STOPPED', 'INSUFFICIEN', 'BY ', 'MR ', 'DRAWERS', 'SIGNATURE','MRS ', 'INSUFFICIENT', 'UTI', 'HDF', 'CBI', 'ANB', 'IND ', 'PNB', 'UTI', 'IDFC', 'BOI', 'KMB', 'CAB', 'YES ','SBI CARDS','TRANSFER OW','CHQ DEP MICR CLG WBO HYDERABAD', 'UNION BANK', 'IRREGULARLY', 'AMOUNT', 'VJB ', 'ANB', 'BOB', 'ICICI', 'SBH', 'CITI', 'VBL', 'IBKL', 'KKBH', 'ANDB', 'CNRB', 'UBIN','TRANSFER ', 'REVERSAL', 'WITHDRAWAL', 'MICR', 'CH ', 'RET ', 'DIFFERS', 'FUNDS', 'RET ', 'PAID ', 'TXN', ' AT ', ' IN ', 'CASH', 'DEP ', 'WDL ', 'BY CASH', 'CAM ', 'ALPHANUMERIC', 'CASH PAID', 'CASH DEPOSIT', 'BRANCH', 'CASH TXN', 'HAND CASH', 'CASH DEP', 'CASH RETURN','BRN ','REFER TO DRAWER',' INDUS ',' AXSK ','DCGBH','ATTN','DC ','AXIR ','XF','KKBKR','COASR','TSABR','IDIBR','RTGSR ','APGBR','AUBLR','KARBR','BARBR','DR ','SK ','HDFC B','HDFC','STATE BAN','UPI','SALARY AC','RBL ','TRANSITORY BRANCH SALARY AC','RETURN','DEUTN','CMS ','URGENT','IDIBH ','BARBH','  TRF','TFR','TRANSFER','IFT','MOB','TPFT','TMBLH','RK S','I W ','NAME DIFFERS ON ADVICE','A CCU RUHIKRPY ','XXXXXXXX','ATM ','CHQ ','MICTS','MC ISSUED','TRTO','BARB ','CHQ DEP MICR CLG WBO','DEPOSIT ','XXXXXX','TPT ','INFT ','FT ','BNA ','CWRO ','DWRO ','HH ','SRY ','TWRO','DEP ','BIL TO ','VPS','ST ','ATW  S AN','ATW ','DD CC ISSUED ','EPC ','RETURN PAY','XX','IOBA','BIL ','SYB','IDB','TDS','CBDT','BULK NEFT','UBI','AXI','CHEQUE','OTHERS','NA ','IMPS','MS','EXCEEDS ARRANGEMENTS','KINDLY','CONTACT','WRONGLY','DELIVERED','INWARD','TSABR','IDIBR','APGBR','KARBR','BARBR','EXCEEDS ARRANGEME','DRAWER','REASONS','CONNEC','TAX',' OWN ','REFER TO DRAWER/N','TRANSITORY ACCOUNT','BULK','RBI','SD ','SAA ','SDL ','SM ','TRFR ']
            new_list=[]
            for i in tqdm(df['Description_clean'][0:]):
                x=i
                for word in stops:
                    x=str(x).replace(word,'')
                    x.strip()
                new_list.append(x)
            final_list=[]
            for k in new_list:
                k.strip()
                final_list.append(k)
                
            df['EXTRACTED ENTITIES']=final_list
            df['EXTRACTED ENTITIES'] = df['EXTRACTED ENTITIES'].apply(str)
            df['EXTRACTED ENTITIES']=df['EXTRACTED ENTITIES'].str.strip()
            df['Account Number'] = df['Account Number'].astype(str)
            df['Account Number']=df['Account Number'].str.strip()
            df['Transaction Mode'] = df['Transaction Mode'].astype(str)
            df['Transaction Mode']=df['Transaction Mode'].str.strip()
            
         
            df['EXTRACTED ENTITIES']=df['EXTRACTED ENTITIES'].fillna('NaN')
            
            df=df.drop(labels='Description_clean',axis=1)
            df['credit'] = df['credit'].apply(lambda x: replace_fun(x))
            df['debit'] = df['debit'].apply(lambda x: replace_fun(x))
            df['debit'] = np.where(df['debit']=='', np.NAN, df['debit'])
            df['credit'] = np.where(df['credit']=='', np.NAN, df['credit'])
            df['debit'] = df['debit'].astype('float')
            df['credit'] = df['credit'].astype('float')
            
            df1=df.groupby(['EXTRACTED ENTITIES']).aggregate({'credit':'sum'}).reset_index()
            df1=df1.sort_values(by='credit',ascending=False)
            df1=df1.reset_index()
            df1=df1.drop(['index'],axis=1)
            
            df2=df.groupby(['EXTRACTED ENTITIES'])['debit'].agg('sum').reset_index()
            df2=df2.sort_values(by='debit',ascending=False)
            df2=df2.reset_index()
            df2=df2.drop(['index'],axis=1)
            
            df3=df.groupby(['Account Number'])['credit'].agg('sum').reset_index()
            df3=df3.sort_values(by='credit',ascending=False)
            df3=df3.reset_index()
            df3=df3.drop(['index'],axis=1)
            
            df4=df.groupby(['Account Number'])['debit'].agg('sum').reset_index()
            df4=df4.sort_values(by='debit',ascending=False)
            df4=df4.reset_index()
            df4=df4.drop(['index'],axis=1)
            
            df5=df.groupby(['Transaction Mode'])['debit'].agg('sum').reset_index()
            df5=df5.sort_values(by='debit',ascending=False)
            df5=df5.reset_index()
            df5=df5.drop(['index'],axis=1)
            
            df6=df.groupby(['Transaction Mode'])['credit'].agg('sum').reset_index()
            df6=df6.sort_values(by='credit',ascending=False)
            df6=df6.reset_index()
            df6=df6.drop(['index'],axis=1)
            
            
#             df['SR.NO'] = np.arange(len(df1))
            df1['SR.NO'] = np.arange(len(df1))
            df2['SR.NO'] = np.arange(len(df2))
            df3['SR.NO'] = np.arange(len(df3))
            df4['SR.NO'] = np.arange(len(df4))
            df5['SR.NO'] = np.arange(len(df5))
            df6['SR.NO'] = np.arange(len(df6))
            
            
            df1=df1[['SR.NO','EXTRACTED ENTITIES','credit']]
            df2=df2[['SR.NO','EXTRACTED ENTITIES','debit']]
            df3=df3[['SR.NO','Account Number','credit']]
            df4=df4[['SR.NO','Account Number','debit']]
            df5=df5[['SR.NO','Transaction Mode','debit']]
            df6=df6[['SR.NO','Transaction Mode','credit']]
            
            
            
            path="C:\\Users\\SAL005\\Desktop\\FUND_FLOW_HOSTED\\data_folder\\{}.xlsx".format(filename.split('.')[0])
            
            writer=pd.ExcelWriter(path,engine='xlsxwriter')
            df.to_excel(writer,sheet_name='Master Sheet',index=False)
            df1.to_excel(writer,sheet_name='Top Credit',index=False)
            df2.to_excel(writer,sheet_name='Top Debit',index=False)
            df3.to_excel(writer,sheet_name='Top Account Credit',index=False)
            df4.to_excel(writer,sheet_name='Top Account Debit',index=False)
            df5.to_excel(writer,sheet_name='Top Transaction mode debit',index=False)
            df6.to_excel(writer,sheet_name='Top Transaction mode credit',index=False)
            writer.close()
            
            
            return send_file(path,attachment_filename=f"{filename.split('.')[0]}.xlsx", mimetype='xlsx', as_attachment=True)
            
            
    return render_template('index_2.html')
                  

if __name__ == "__main__":
    app.run(debug=True, host='172.20.1.81', use_reloader = False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://172.20.1.81:5000/ (Press CTRL+C to quit)
172.20.1.81 - - [17/Jul/2023 15:48:15] "GET / HTTP/1.1" 200 -
172.20.1.81 - - [17/Jul/2023 15:48:15] "GET /static/css/bootstrap.min.css HTTP/1.1" 404 -
172.20.1.81 - - [17/Jul/2023 15:48:36] "GET /favicon.ico HTTP/1.1" 404 -
0it [00:00, ?it/s]C:\Users\SAL005\AppData\Local\Temp\ipykernel_15520\3681235050.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Transaction Mode'].iloc[idx]='IMPS'
1it [00:01,  1.36s/it]C:\Users\SAL005\AppData\Local\Temp\ipykernel_15520\3681235050.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Transaction Mode'].iloc[i